# IMPORTING LIBRARIES

In [39]:
import pandas as pd
import numpy as np
import imblearn
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import linear_model
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report, confusion_matrix

# READING CSV FILE

In [40]:
data = pd.read_csv("./creditcard.csv")
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
# now let us check in the number of Percentage
Count_Normal_transacation = len(data[data["Class"]==0]) # normal transaction are repersented by 0
Count_Fraud_transacation = len(data[data["Class"]==1]) # fraud by 1
Percentage_of_Normal_transacation = Count_Normal_transacation/(Count_Normal_transacation+Count_Fraud_transacation)
print("percentage of normal transacation is",Percentage_of_Normal_transacation*100)
Percentage_of_Fraud_transacation= Count_Fraud_transacation/(Count_Normal_transacation+Count_Fraud_transacation)
print("percentage of fraud transacation",Percentage_of_Fraud_transacation*100)


percentage of normal transacation is 99.82725143693798
percentage of fraud transacation 0.1727485630620034


In [42]:
print("length of training data",len(data))
print("length of normal data",len(data[data["Class"]==0]))
print("length of fraud  data",len(data[data["Class"]==1]))

length of training data 284807
length of normal data 284315
length of fraud  data 492


In [43]:
X = data[['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19',
          'V20','V21','V22','V23','V24','V25','V26','V27','V28','Amount']]
Y = data['Class']

# Oversampling the data

In [44]:
from imblearn.over_sampling import SMOTE

os = SMOTE(random_state=0) 
columns = X.columns

os_data_X,os_data_y=os.fit_sample(X,Y)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=["Class"])
# we can Check the numbers of our data
print("length of oversampled data is ",len(os_data_X))
print("Number of normal transcation in oversampled data",len(os_data_y[os_data_y["Class"]==0]))
print("No.of fraud transcation",len(os_data_y[os_data_y["Class"]==1]))
print("Proportion of Normal data in oversampled data is ",len(os_data_y[os_data_y["Class"]==0])/len(os_data_X))
print("Proportion of fraud data in oversampled data is ",len(os_data_y[os_data_y["Class"]==1])/len(os_data_X))

length of oversampled data is  568630
Number of normal transcation in oversampled data 284315
No.of fraud transcation 284315
Proportion of Normal data in oversampled data is  0.5
Proportion of fraud data in oversampled data is  0.5


In [45]:
os_data_X

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.620000
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.690000
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.660000
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.500000
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.990000
5,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,-0.371407,...,0.084968,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.670000
6,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,-0.099254,...,-0.219633,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.990000
7,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,1.249376,...,-0.156742,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,40.800000
8,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,-0.410430,...,0.052736,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,93.200000
9,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,-0.366846,...,0.203711,-0.246914,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,3.680000


# SPLITTING THE DATASET INTO TRAIN AND TEST

In [46]:
X_over = os_data_X[['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19',
               'V20','V21','V22','V23','V24','V25','V26','V27','V28','Amount']]
y_over = os_data_y['Class']

In [47]:

X_over_train, X_over_test, y_over_train, y_over_test = train_test_split(X_over,y_over,test_size = 0.3, random_state = 0)

# RFE ON LOGISTIC REGRESSION

In [48]:
model = LogisticRegression()
rfe = RFE(model)
fit = rfe.fit(X_over_train,y_over_train)
print("Number of Features: %d"% fit.n_features_)
print("Selected features: %s"% fit.support_)
print("Ranking of features: %s"% fit.ranking_)  

KeyboardInterrupt: 

# MODEL FITTING ON TRAIN DATA

In [11]:
X = X_over_train[['V4','V8','V9','V10','V13','V14','V16','V20','V21','V22','V24','V25','V26','V27']]

In [12]:
classifier = LogisticRegression(random_state = 0)
classifier.fit(X, y_over_train)

y_pred = classifier.predict(X)
print(confusion_matrix(y_over_train,y_pred))  
print(classification_report(y_over_train,y_pred)) 
print(accuracy_score(y_over_train, y_pred))

[[194683   4460]
 [ 15578 183320]]
             precision    recall  f1-score   support

          0       0.93      0.98      0.95    199143
          1       0.98      0.92      0.95    198898

avg / total       0.95      0.95      0.95    398041

0.9496584522699922


## Test data accuracy

In [13]:
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_over_test, y_over_test)
y_pred = classifier.predict(X_over_test)
print(confusion_matrix(y_over_test,y_pred))  
print(classification_report(y_over_test,y_pred)) 
print(accuracy_score(y_over_test, y_pred))

[[83628  1544]
 [ 5536 79881]]
             precision    recall  f1-score   support

          0       0.94      0.98      0.96     85172
          1       0.98      0.94      0.96     85417

avg / total       0.96      0.96      0.96    170589

0.9584967377732445


## Grid search on train data

In [15]:
# Logistic regression
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

logregpipe = Pipeline([('scale', StandardScaler()),
                   ('logreg',LogisticRegression(multi_class="multinomial",solver="lbfgs"))])

# Gridsearch to determine the value of C
param_grid = {'logreg__C':np.arange(0.01,100,10)}
logreg_cv = GridSearchCV(logregpipe,param_grid,cv=5,return_train_score=True)
logreg_cv.fit(X,y_over_train)
print(logreg_cv.best_params_)

bestlogreg = logreg_cv.best_estimator_
bestlogreg.fit(X,y_over_train)
bestlogreg.coef_ = bestlogreg.named_steps['logreg'].coef_
bestlogreg.score(X,y_over_train)

{'logreg__C': 10.01}


0.9496710137900367

## Grid search on test data

In [18]:
bestlogreg = logreg_cv.best_estimator_
bestlogreg.fit(X_over_test,y_over_test)
bestlogreg.coef_ = bestlogreg.named_steps['logreg'].coef_
bestlogreg.score(X_over_test,y_over_test)

0.9585319100293689

## Cross Validation on train data

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import cross_validation

predicted = cross_validation.cross_val_predict(LogisticRegression(), X, y_over_train, cv=10)
print (metrics.accuracy_score(y_over_train, predicted))


C:\Users\sayal\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0.9496835753100811


# Cross Validation on test data

In [20]:
predicted1 = cross_validation.cross_val_predict(LogisticRegression(), X_over_test, y_over_test, cv=10)

print (metrics.classification_report(y_over_test, predicted1))
print (metrics.accuracy_score(y_over_test, predicted1))


             precision    recall  f1-score   support

          0       0.94      0.98      0.96     85172
          1       0.98      0.94      0.96     85417

avg / total       0.96      0.96      0.96    170589

0.9584322553036831


# K BEST ON LOGISTIC REGRESSION

In [24]:
#array = under_sample.values
test = SelectKBest(score_func=f_classif, k=10)
fit = test.fit(X_over_train, y_over_train)
print("scores_:",test.scores_)
print("pvalues_:",test.pvalues_)
print("selected index:",test.get_support(True))
print("after transform:",test.transform(X_over_train)) 

scores_: [8.90094295e+04 1.34490388e+05 1.88238248e+05 4.20919510e+05
 7.01624088e+04 8.27595776e+04 1.17168375e+05 2.23421604e+03
 2.02211994e+05 2.80165278e+05 3.89901322e+05 3.69625988e+05
 1.92694387e+03 5.77083297e+05 5.09794479e+02 2.29924455e+05
 1.94388512e+05 1.19822984e+05 3.01968294e+04 1.46847404e+04
 7.76561530e+03 1.42699446e+01 6.32743066e+01 4.76825994e+03
 1.19644623e+03 1.45353018e+03 4.46225074e+03 4.20967142e+03
 1.73454612e+03]
pvalues_: [0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 8.28014570e-113 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 1.58396646e-004 1.80301758e-015 0.00000000e+000
 8.87098339e-262 1.84523094e-317 0.00000000e+000 0.00000000e+000
 0.00000000e+000]
selected index: [ 1  2  3  8  9 10 11 13 15 16]
after transform:

# MODEL FITTING ON TRAIN DATA

In [26]:
logreg = LogisticRegression()
## fitiing the model
X = X_over_train[['V3','V4','V7','V9','V10','V11','V12','V14','V16','V17']]
logreg.fit(X, y_over_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [28]:
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_over_train, y_over_train)
y_pred = classifier.predict(X_over_train)
print(confusion_matrix(y_over_train,y_pred))  
print(classification_report(y_over_train,y_pred)) 
print(accuracy_score(y_over_train, y_pred))

[[195726   3417]
 [ 12514 186384]]
             precision    recall  f1-score   support

          0       0.94      0.98      0.96    199143
          1       0.98      0.94      0.96    198898

avg / total       0.96      0.96      0.96    398041

0.9599764848344768


# MODEL FITTING ON TEST DATA

In [30]:
## fitiing the model
X2 = X_over_test[['V2','V3','V4','V9','V10','V11','V12','V14','V16','V17']]
# logreg.fit(X2, y_under_test)

In [31]:

classifier = LogisticRegression(random_state = 0)
classifier.fit(X2, y_over_test)
y_pred = classifier.predict(X2)
print(confusion_matrix(y_over_test,y_pred))  
print(classification_report(y_over_test,y_pred)) 
print(accuracy_score(y_over_test, y_pred))


[[83454  1718]
 [ 6623 78794]]
             precision    recall  f1-score   support

          0       0.93      0.98      0.95     85172
          1       0.98      0.92      0.95     85417

avg / total       0.95      0.95      0.95    170589

0.9511047019444395


## Grid Search on train data

In [32]:
# Logistic regression
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

logregpipe = Pipeline([('scale', StandardScaler()),
                   ('logreg',LogisticRegression(multi_class="multinomial",solver="lbfgs"))])

# Gridsearch to determine the value of C
param_grid = {'logreg__C':np.arange(0.01,100,10)}
logreg_cv = GridSearchCV(logregpipe,param_grid,cv=5,return_train_score=True)
logreg_cv.fit(X,y_over_train)
print(logreg_cv.best_params_)

bestlogreg = logreg_cv.best_estimator_
bestlogreg.fit(X,y_over_train)
bestlogreg.coef_ = bestlogreg.named_steps['logreg'].coef_
bestlogreg.score(X,y_over_train)

{'logreg__C': 30.01}


0.9544469037109242

# Grid search on test data

In [33]:
bestlogreg = logreg_cv.best_estimator_
bestlogreg.fit(X2,y_over_test)
bestlogreg.coef_ = bestlogreg.named_steps['logreg'].coef_
bestlogreg.score(X2,y_over_test)

0.9511047019444395

## Cross Validation on train data

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import cross_validation

predicted = cross_validation.cross_val_predict(LogisticRegression(), X, y_over_train, cv=10)
print (metrics.accuracy_score(y_over_train, predicted))


0.9544418791029065


# Cross validation on test data

In [35]:
predicted1 = cross_validation.cross_val_predict(LogisticRegression(), X_over_test, y_over_test, cv=10)

print (metrics.classification_report(y_over_test, predicted1))
print (metrics.accuracy_score(y_over_test, predicted1))


             precision    recall  f1-score   support

          0       0.94      0.98      0.96     85172
          1       0.98      0.94      0.96     85417

avg / total       0.96      0.96      0.96    170589

0.9584322553036831


# PCA ON LOGISTIC REGRESSION

In [58]:
pca = PCA(n_components=5)
X_under_train = pca.fit_transform(X_under_train)
X_under_test = pca.transform(X_under_test)
explained_variance = pca.explained_variance_ratio_
print(explained_variance)

[9.93486071e-01 4.51088486e-03 5.49623610e-04 5.22069220e-04
 2.47273720e-04]


# MODEL FITTING ON TRAIN & TEST DATA

In [59]:
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_under_train, y_under_train)
y_pred = classifier.predict(X_under_test)
print(confusion_matrix(y_under_test,y_pred))  
print(classification_report(y_under_test,y_pred)) 
print(accuracy_score(y_under_test, y_pred))

[[147   2]
 [ 16 131]]
             precision    recall  f1-score   support

          0       0.90      0.99      0.94       149
          1       0.98      0.89      0.94       147

avg / total       0.94      0.94      0.94       296

0.9391891891891891


## Grid Search on test & train data

In [37]:
# Logistic regression
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

logregpipe = Pipeline([('scale', StandardScaler()),
                   ('logreg',LogisticRegression(multi_class="multinomial",solver="lbfgs"))])

# Gridsearch to determine the value of C
param_grid = {'logreg__C':np.arange(0.01,100,10)}
logreg_cv = GridSearchCV(logregpipe,param_grid,cv=5,return_train_score=True)
logreg_cv.fit(X_over_train,y_over_train)
print(logreg_cv.best_params_)

bestlogreg = logreg_cv.best_estimator_
bestlogreg.fit(X_over_train,y_over_train)
bestlogreg.coef_ = bestlogreg.named_steps['logreg'].coef_
bestlogreg.score(X_over_train,y_over_train)

{'logreg__C': 60.01}


0.9600317555226723

# Cross validation on test & train data

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import cross_validation

predicted = cross_validation.cross_val_predict(LogisticRegression(),X_over_train,y_over_train, cv=10)
print (metrics.accuracy_score(y_over_train, predicted))

0.9599965832665479
